#Micro Trainning: Topic Modelling

##Use Case

Imagine we have 1000 documents without a catelog to classify them into any catgories or interesting categories

In [8]:
# Grab my Tools
import os
import utils
import json
from collections import Counter
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore, TfidfModel
import pyLDAvis.gensim

# Data Path
path = os.path.join(os.getcwd(), 'data')

# Enable visualization
pyLDAvis.enable_notebook()

#### Steps to Model topics
* Reading Data into Machine Understandable Format
* Cleaning
* Building a Word Frequency Dictonary 
* Fitting a [LDA](https://ai.stanford.edu/~ang/papers/jair03-lda.pdf) ([for dummies](https://www.quora.com/What-is-a-good-explanation-of-Latent-Dirichlet-Allocation))
* Visualizing Results
* Building on Top of Result

In [3]:
# Reading Data
files = utils.readfiles(path)
lst_files = utils.parsebibfiles(files)

# Sample
print("Read {0} files\nSample Entry:\n {1}".format(len(lst_files), json.dumps(lst_files[1], indent=2, sort_keys=True)))

Read 1485 files
Sample Entry:
 {
  "ENTRYTYPE": "article",
  "ID": "Boci2015",
  "abstract": "The first building block of the Federal Aviation Administration's (FAA) Next Generation Air Transportation System (NextGen) initiative to modernize the US national airspace system (NAS) was the implementation of the Automatic Dependent Surveillance-Broadcast (ADS-B) ground infrastructure. A primary aspect of the ADS-B program design is the terrestrial radio station infrastructure. It determined the terrestrial radio stations layout throughout the US and was optimized to meet system performance, safety and security in the NAS. In March 2014, the FAA completed the nationwide infrastructure upgrade, enabling air traffic controllers to track aircraft with greater accuracy and reliability, while giving pilots more information in the cockpit. More than 650 ADS-B radios communicate with equipped aircraft, supporting the new satellite-based surveillance system. Currently, the ADS-B system ingests proc

In [4]:
# Clean Data
lst_clean_bow = [utils.clean_abstract(file) for file in lst_files]

# Sample
print("Cleaned Text\nSample\nTop 10 Words in Document\n{0}".format(Counter(lst_clean_bow[2]).most_common(10)))

Cleaned Text
Sample
Top 10 Words in Document
[('data', 5), ('research', 4), ('databas', 3), ('meet', 2), ('impact', 2), ('report', 2), ('divers', 2), ('manag', 2), ('everi', 1), ('year', 1)]


In [5]:
# build gensim dict
corpus_dict = Dictionary(lst_clean_bow)

# filter low freq words threshold > 15 Abstracts
corpus_dict.filter_extremes(no_below=2)

# gensim doc2bow
bow_corpus = [corpus_dict.doc2bow(doc) for doc in lst_clean_bow]

# tf-idf model over bow
tfidf_model = TfidfModel(bow_corpus)
corpus_tfidf = tfidf_model[bow_corpus]




In [11]:
# gensim LDA model over tf-idf
lda_tfidf_model = LdaMulticore(corpus_tfidf, num_topics=10, id2word=corpus_dict, passes=2, workers=4)

# preview
for idx, topic in lda_tfidf_model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

for index, score in sorted(lda_tfidf_model[bow_corpus[12]], key=lambda tup: -1 * tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_tfidf_model.print_topic(index, 10)))

# visualization
vis = pyLDAvis.gensim.prepare(topic_model=lda_tfidf_model, corpus=bow_corpus, dictionary=corpus_dict)
vis


Topic: 0 Word: 0.003*"vr" + 0.002*"train" + 0.002*"simul" + 0.002*"control" + 0.002*"work" + 0.002*"design" + 0.002*"process" + 0.002*"industri" + 0.002*"technolog" + 0.002*"data"
Topic: 1 Word: 0.003*"design" + 0.003*"mainten" + 0.002*"simul" + 0.002*"factori" + 0.002*"comput" + 0.002*"model" + 0.002*"assembl" + 0.002*"technolog" + 0.002*"digit" + 0.002*"inform"
Topic: 2 Word: 0.002*"vr" + 0.002*"assembl" + 0.002*"design" + 0.002*"train" + 0.002*"model" + 0.002*"digit" + 0.002*"engin" + 0.002*"plan" + 0.002*"learn" + 0.002*"factori"
Topic: 3 Word: 0.004*"mine" + 0.004*"safeti" + 0.003*"train" + 0.003*"robot" + 0.003*"design" + 0.003*"simul" + 0.002*"coal" + 0.002*"tool" + 0.002*"vr" + 0.002*"interact"
Topic: 4 Word: 0.002*"design" + 0.002*"model" + 0.002*"prototyp" + 0.002*"augment" + 0.002*"ar" + 0.002*"industri" + 0.002*"manufactur" + 0.002*"game" + 0.002*"technolog" + 0.002*"vr"
Topic: 5 Word: 0.003*"train" + 0.003*"ar" + 0.002*"manufactur" + 0.002*"design" + 0.002*"simul" + 0.002*

In [10]:
# Visualize
# var = utils.visualize_lda_model(model=lda_tfidf_model, corpus=bow_corpus, corpus_dict=corpus_dict)
# print(var)
vis = pyLDAvis.gensim.prepare(topic_model=lda_tfidf_model, corpus=bow_corpus,dictionary=corpus_dict)
vis

NameError: name 'lda_tfidf_model' is not defined